In [3]:
import pandas as pd
import numpy as np

In [4]:
bits_info = pd.DataFrame({
    "DRILL_BIT_NAME":["Buzz Drilldrin", "AstroBit", "Apollo", "ChallengDriller"],
    "CPR": [5000,3000,1000,10000],
    "CPF": [1.5,1,4,0],
    "CPH":[0,1500,2500,0]
})
bits_info

,DRILL_BIT_NAME,CPR,CPF,CPH
0,Buzz Drilldrin,5000,1.5,0
1,AstroBit,3000,1.0,1500
2,Apollo,1000,4.0,2500
3,ChallengDriller,10000,0.0,0


In [56]:
csv = pd.read_csv("./Cleaned/Asteroid 1 - clean.csv")
numeric_cols = csv.loc[:, csv.columns != 'DRILL_BIT_NAME']
numeric_cols.loc[:][numeric_cols < 0] = np.NAN                         
numeric_cols["DRILL_BIT_NAME"] = csv["DRILL_BIT_NAME"]
data = numeric_cols.dropna()


In [57]:
aggregated = (
    data
    .groupby(['DRILL_BIT_ID','DRILL_BIT_NAME'])
    .agg({'BIT_DEPTH': ['min', 'max'], 'RATE_OF_PENETRATION' : ['mean']})
    .pipe(lambda x: x.set_axis(x.columns.map('_'.join), axis=1))
)

aggregated.reset_index(inplace=True)

aggregated["total_depth"] = aggregated["BIT_DEPTH_max"] - aggregated["BIT_DEPTH_min"]
aggregated["total_hours"] = aggregated["total_depth"] / aggregated["RATE_OF_PENETRATION_mean"]
aggregated = aggregated.merge(bits_info)

# Do your computation
aggregated["cost"] = aggregated['CPR'] + aggregated["total_depth"] * aggregated['CPF'] + aggregated["total_hours"] * aggregated['CPH']
aggregated

,DRILL_BIT_ID,DRILL_BIT_NAME,BIT_DEPTH_min,BIT_DEPTH_max,RATE_OF_PENETRATION_mean,total_depth,total_hours,CPR,CPF,CPH,cost
0,1,Apollo,117.30,9316.52,356.296308,9199.22,25.819016,1000,4.0,2500,102344.419519
1,2,Buzz Drilldrin,9317.01,9328.54,118.200000,11.53,0.097547,5000,1.5,0,5017.295000
2,3,Buzz Drilldrin,9329.03,9342.52,117.457143,13.49,0.114850,5000,1.5,0,5020.235000
3,4,Buzz Drilldrin,9343.07,10510.51,110.780891,1167.44,10.538280,5000,1.5,0,6751.160000
4,5,Buzz Drilldrin,10511.04,15323.00,108.802347,4811.96,44.226619,5000,1.5,0,12217.940000


In [47]:
aggregated['cost'].sum()

104312.52190875287